#  🎮 Actieplanning
📚 [^hf]
[^hf]: Deze sectie is gebaseerd op de [Hugging Face Deep RL Course](https://huggingface.co/learn/deep-rl-course/en/unit0/introduction).

:::{iframe} https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit0/huggy.mp4
:width: 100%
:::

## _Reinforcement Learning_

Zoals we reeds zagen in de bespreking van [ML taken](../basics/tasks.ipynb), is het bij {ref}`actieplanning <target-planning>` de bedoeling om optimale actiesequenties te leren, met het oog op het bereiken van een bepaald doel. Hierbij wordt gewerkt met een specifieke vorm van [ervaring](../basics/experience.ipynb): _{ref}`reinforcement learning <target-reinforcement-learning>`_ (RL). In tegenstelling tot _(self-)supervised learning_, wordt er bij RL in de zoektocht naar optimale parameters **geen rechtstreekse feedback** gegeven over de juistheid van een individuele actie. In tegenstelling tot _unsupervised learning_, wordt het leren gestuurd door op zoek te gaan naar een **zo groot mogelijke cumulatieve beloning uit de omgeving**.

:::{important}
(target-rl-agent)=
RL is dus een specifiek principe waarmee een model [ervaring](../basics/experience.ipynb) kan opdoen dat vooral gebruikt wordt bij {ref}`actieplanning <target-planning>`. principe. **RL verwijst dus {u}`niet` naar een specifiek {ref}`model <target-model>`**! Zo zagen we reinforcement al onder de vorm van _reinforcement learning from human feedback_ (RLHF) in de context van het finetunen van large language modellen. In deze secties bespreken we de benadering specifiek in het kader van {ref}`actieplanning <target-planning>`. Daarbij wordt vaak de term **"_agent_"** gebruikt. Dit verwijst algemeen naar het systeem dat **iteratief observeert, acties neemt en resultaten van acties integreert**. De agent kan daarbij rekenen op de voorspellingen van **één of meerdere ML modellen**.
:::

_Agents_ die via RL getraind worden, hebben de volgende algemene eigenschappen:
- **Iteratieve output**: De _agent_ bepaalt niet in één keer de hele sequentie van acties, maar registreert en integreert na iedere actie feedback over de gevolgen van die actie in de omgeving in een nieuwe (voorspelde) actie.
- **Trial-and-error learning**: Leren gebeurt door acties uit te proberen en de uiteindelijke beloning te observeren.
- **Markov-eigenschap**: Dit is een centrale aanname: om te weten wat de beste volgende actie is, heeft de _agent_ enkel kennis nodig van de huidige stand van zaken in de omgeving (_state_). In een schaakspel, bijvoorbeeld, worden acties enkel genomen op basis van reflecties over de toekomst, gegeven de huidige staat op het bord, niet op basis van reflecties over hoe de huidige staat tot stand kwam.
:::

## RL Loop
(target-rl-loop)=
Het RL proces moet gezien worden als een continue loop van interacties tussen een _agent_ en een omgeving:

[![](https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit1/RL_process_game.jpg)](https://huggingface.co/learn/deep-rl-course/en/unit0/introduction)

De cyclus werkt als volgt:
1. De _agent_ observeert de huidige _state_ $S_t$ van de omgeving
2. Op basis van die state kiest de agent een actie $A_t$
3. Dit geeft aanleiding tot een een nieuwe _state_ $S_{t+1}$ in de omgeving
4. Of basis van die verandering, krijgt de _agent_ een _reward_ $R_{t+1}$ signaal uit de omgeving

**Het doel van de agent is om de _cumulatieve reward_ over tijd te maximaliseren**.

[![](https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit1/sars.jpg)](https://huggingface.co/learn/deep-rl-course/en/unit0/introduction)

## Kernconcepten
### _Omgeving_
(target-rl-env)=
Dit is de "wereld" waarin de _agent_ zich begeeft.

### _States_
(target-rl-state)=
Een **_state_** is een momentane beschrijving van de omgeving.
De _state_ is hetzij {u}`volledig geobserveerd` (bv. bij een schaakbord),

[![](https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit1/chess.jpg)](https://huggingface.co/learn/deep-rl-course/en/unit0/introduction)

hetzij {u}`gedeeltelijk geobserveerd` (bv. bij een videogame zoals Super Mario).

[![](https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit1/mario.jpg)](https://huggingface.co/learn/deep-rl-course/en/unit0/introduction)

In de praktijk onderscheiden we daarom:
- **_State_** $s_t$: De complete beschrijving van de omgeving op een bepaald moment $t$.
(target-rl-observation)=
- **Observatie** $o_t$: Het gedeelte van de omgeving dat de agent waarneemt op moment $t$.

### Acties
(target-rl-action-space)=
De **_action space_** bevat alle mogelijke acties die een agent kan uitvoeren:
- Discrete _action space_: Eindig aantal acties (bijv. Super Mario: links, rechts, springen, bukken)
- Continue _action space_: Oneindig aantal acties (bijv. de stuurhoek bij een zelfrijdende wagen)

### _Rewards_
(target-rl-reward)=
De **_reward_** of beloning is het enige feedbacksignaal voor de agent waaruit het tijdens de training de optimale parameters gezocht kunnen worden voor de onderliggende ML modellen. De _reward_ hypothese stelt dat alle doelen kunnen worden beschreven als het maximaliseren van de verwachte cumulatieve _reward_.

(target-rl-return)=
De **cumulatieve _reward_** of **_return_** is gegeven als:

$$
\begin{align}
R(\tau) &= r_{t+1} + r_{t+2} + r_{t+3} + r_{t+4}\ldots \cr
&= \sum_{k=0}^{\infty} r_{t+k+1}
\end{align}
$$

waarbij $\tau$ staat voor het traject: de sequentie van _state_-actieparen tot op moment $t$.

(target-rl-discounting)=
Op basis van de onderliggende ML modellen, kunnen _agents_ (expliciet of impliciet) rekening houden met _expected_ cumulatieve _rewards_ om de meest wenselijke _volgende_ actie te bepalen. Omdat voorspellingen onzekerder zijn naarmate ze verder in de toekomst liggen, wordt bij de berekening van de _return_ een correctie toegepast naarmate de returns verder in de toekomst liggen. Hierdoor kan er bij de keuzes van acties voorkeur gegeven worden aan acties met een meer onmiddellijke winst. Dit gebeurt aan de hand van een **_discount_ factor**, wat resulteert in de **_discounted return_** is:

$$
\begin{align}
R(\tau) &= r_{t+1} + \gamma r_{t+2} + \gamma^2 r_{t+3} + \gamma^3 r_{t+4} + \ldots \cr
&= \sum_{k=0}^{\infty} \gamma^k r_{t+k+1}
\end{align}
$$

waarbij $\gamma$ (gamma) de **_discount_ factor** is ($0 \leq \gamma \leq 1$):
- $\gamma$ dicht bij 1: Agent houdt rekening met lange-termijn rewards
- $\gamma$ dicht bij 0: Agent focust op korte-termijn rewards

### _Value_ functie
(target-rl-value-function)=
De _value_ functie is de functie die de verwachte _discounted return_ bepaalt gegeven een bepaalde _state_ ($V$-functie) of _state_-actiepaar ($Q$-functie).

### Policy
(target-rl-policy)=
De **policy** $\pi$ is de strategie van de agent - het bepaalt welke actie de agent kiest in een bepaalde state:

$$
\pi(a|s) = P(A_t = a | S_t = s)
$$

Dit kan zijn:
- Deterministisch: $a = \pi(s)$ - altijd dezelfde actie voor een state
- Stochastisch: $\pi(a|s)$ - kansverdeling over acties

## Modellen

Het **doel van de agent is om tijdens de trainingsfase de optimale policy** $\pi^*$ te vinden die de verwachte return maximaliseert. Dit kan op twee manieren:

(target-rl-policy-based)=
(target-rl-value-based)=
1. **_Policy-based_**: Er wordt een ML model getraind met _states_ als input en acties als output. Daaruit krijgt de _agent_ bij _inference_ dus rechtstreeks informatie over de optimale actie.
2. **_Value based_**: Een ML model leert de _return_ te voorspellen, voor verschillende _states_ of _state_-actieparen. Daarna kan de _agent_, gegeven een bepaalde _state_, bijvoorbeeld de actie kiezen met de grootste verwachte _return_ (dit is een zogenaamde _greedy policy_).

In principe kan ieder type ML model hierin gekoppeld worden. In wat volgt zullen we echter focussen op toepassingen met _deep learning_ modellen of zogenaamde _Deep RL_. De beschrijving van deze modellen, in termen van architectuur, parameters, features en hyperparameters wordt bepaald door de use-case en heeft geen specifieke eigenschappen die gekoppeld zijn an het domein van actieplanning.

## Leeralgoritmes

De training is bij RL doorgaans veel complexer dan bij andere domeinen van ML. Dat komt vooral door de verschillende leeralgoritmes, hyperparameters (bv. om exploratie versus exploitatie te bepalen) en complexe _loss_ functies om de _reward_ signalen te vertalen naar de updates voor de modelparameters. Door de grote **variabiliteit**, bestaat er een reëel **risico op instabiele training**. Die variabiliteit komt uiteraard ook uit de omgeving van de agent zelf die soms heel complex kunnen zijn (denk bv. aan een zelfrijdende wagen). Hoe krachtig de resultaten van RL ook kunnen zijn, het toepassen van RL in een nieuwe omgeving blijft doorgaans heel uitdagend.  
  
De leeralgoritmes hangen nauw samen met de gekozen modeltypes: _policy based_, _value based_ of beide.

### _Q-learning_
(target-rl-q-learning)=
_**Q-Learning**_ is een _value based_ algoritme waarbij een optimale _state_-actie _value_ functie (of model) $Q^*(s, a)$ geleerd wordt. Een _greedy policy_ neemt dan meestal de volgende vorm aan:

$$
\pi^*(s) = \underset{a}{\operatorname{\argmax}} Q^*(s, a)
$$

:::{note} Exploratie versus exploitatie
Tijdens het trainen kan een agent gestuurd worden richting meer of minder exploratie van de $Q$ ruimte. Bij een _greedy policy_ kiest de _agent_ telkens voor de actie met de hoogste verwachte (_discounted_) _return_ ("exploitatie"). Daardoor kan het leertraject traag verlopen omdat de agent niet of te weinig terecht komt in onbekende states die mogelijks tot een hogere _return_ leiden. Daarom kiest men meestal voor een zogenaamde $\epsilon$-_greedy policy_ waarbij met een probabiliteit $\epsilon$ een random actie (exploration) genomen wordt in plaats van de beste actie volgens $Q(s, a)$
:::

Bij _Deep Q Learning_ wordt de zogenaamde $Q$-_loss_ berekend door de voorspelde (_discounted_) return bij een gegeven _state_-actionpaar te vergelijken met een $Q$-_target_ (in de standaard MSE vorm). Die _target_ wordt {u}`geschat` door de voorspellingen van het model zelf (met parameters $\theta$) over de $Q$-_value_ van de volgende state $s'$, samen met de onmiddellijke _return_ $r$, in de zogenaamde {u}`_Bellman_ vergelijking` in te vullen:

$$
Q_{\text{target}} = r + \gamma \underset{a'}{\operatorname{\argmax}} Q(s', a', \theta)
$$

De gradient van de $Q$-_loss_ wordt dan gebruikt om de parameters van het neurale netwerk via standaard _backpropagation_ en _gradient descent_ te updaten.

### _Policy gradient learning_
(target-rl-policy-gradient)=
Deze familie van leeralgoritmes zijn _policy based_. Er wordt dus rechtstreeks een optimale _policy_-functie (of model) ${\pi_{\theta}}^*(a|s)$ geleerd. **_Policy gradient learning_** past _gradient ascent_ toe op de gradient van de verwachtte _return_ (om die dus te maximaliseren):

$$
\begin{align}
\nabla J(\theta) &\approx \mathbb{E}\left[ \nabla \log \pi(a|s; \theta) \cdot G_t \right] \cr
\pmb{\theta}^{k+1} &= \pmb{\theta}^k + \lambda \nabla J(\theta)
\end{align}
$$

Waarbij:
- $\nabla \log \pi(a|s; \theta)$: De richting die de probabiliteit van actie $a$ verhoogt.
- $G_t$: De _return_ (cumulatieve _reward_) vanaf tijdstip $t$.
- Het dot-product: Verhoogt de probabiliteit van acties die tot hoge _returns_ leidden en verlaag de probabiliteit van acties tot lage _returns_ leidden.
  
Dit vormt de basis van het REINFORCE algoritme door {cite}`10.1007/BF00992696`. In recent werk wordt meestal gewerkt met het zogenaamde _{u}`Proximal Policy Optimization`_ (PPO) algoritme wat over het algemeen stabielere resultaten oplevert.  
  
Merk op dat de optimalisatie hier niet op basis van een _loss_ functie gebeurt, maar door rechtstreeks de verwachtte _return_ $J(\theta)$ te maximaliseren[^ascent].

[^ascent]: Vandaar dat we hier te maken hebben met _gradient ascent_ en niet _gradient descent_.

### _Actor-critic learning_
(target-rl-actor-critic)=
Hierbij wordt een _value_-functie gecombineerd met een _policy_-functie. De _actor_ leert de _policy_ ${\pi_{\theta}}^*(a|s)$ terwijl de _critic_ de _value_-functie $Q^*(s, a)$ of $V^*(s)$ leert. In plaats van de werkelijke _returns_ $G_t$ te gebruiken bij het updaten van de policy (zoals bij _policy gradient learning_), worden _value_-predicties van de _critic_ gebruikt (_expected return_). Net zoals het PPO algoritme, leidt deze techniek over het algemeen tot stabielere resultaten. Doordat er niet moet gewacht worden tot tot het einde van een interactie om de werkelijke _return_ te berekenen, wordt deze benadering vaak gekozen omwille van _sample_ efficiëntie en bij continue taken (bv. robotica; zie verder). De meest populaire variant _{u}`Soft-Actor-Critic`_ (SAC) introduceert ook entropie in de parameter-optimalisatie functie ({cite}`10.48550/arXiv.1801.01290`). 

## Taken
We maken een onderscheid tussen twee soorten taken afhankelijk of er sprake is van een eind-_state_.

### Episodische taken
Hierbij is er een duidelijke eind-_state_, bijvoorbeeld, wanneer een game uitgespeeld is.
### Continue taken
Hierbij is er geen eindpunt. Denk aan autonoom rijden, robotica, _trading_, enz.

[![](https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit1/stock.jpg)](https://huggingface.co/learn/deep-rl-course/en/unit1/tasks)

RL is geschikt voor problemen waarbij:
- sequentiële beslissingen cruciaal zijn,
- interactie met omgeving mogelijk is,
- feedback of wenselijk of onwenselijke predicties (_rewards_) pas na verloop van tijd duidelijk worden en
- geen expliciete labels beschikbaar zijn,

zoals bij:
- 🎮 Game AI: AlphaGo, Dota 2, StarCraft II
- 🤖 Robotica: Lopen, grijpen, manipulatie
- 🚗 Autonome systemen: Zelfrijdende auto's
- 💰 Trading: Automatische handelssystemen
- 🏭 Resource management: Datacenter koeling, energie-optimalisatie
- 💊 Gezondheidszorg: Behandelprotocollen, medicatie timing
- 🎯 Aanbevelingssystemen: Personalisatie over de tijd

## Evaluatie
Bij RL evalueren we op basis van scores zoals:
- _Average return_: De gemiddelde cumulatieve _reward_ over meerdere episodes.
- _Success rate_: Het percentage van de episodes waarin een specifiek doel wordt bereikt.
- Leercurve: Hoe snel verbetert de agent tijdens training?
- enz.  
 
Dit is anders dan typische scores zoals accuracy bij klassificatie of MSE bij regressie, omdat we geïnteresseerd zijn in de {u}`prestaties over de tijd`.


## Voordelen
- Geen gelabelde data nodig: RL leert rechtstreeks uit interactie met de omgeving, zonder vooraf gelabelde voorbeelden
- Sequentiële besluitvorming: Ideaal voor problemen waar acties gevolgen hebben op toekomstige situaties
- Adaptief leren: Agents kunnen zich aanpassen aan veranderende omgevingen en nieuwe situaties
- Autonomie: Agents ontdekken zelf strategieën die mensen misschien niet hadden bedacht (bv. AlphaGo)
- Breed toepassingsgebied: Van games en robotica tot resource management en gezondheidszorg
- Trial-and-error: Natuurlijke leerwijze die exploratie en experimenteren mogelijk maakt

## Nadelen
- Sample inefficiëntie: Vaak miljoenen interacties nodig, zeker bij complexe omgevingen
- _Reward_ design: Vaak moeilijk om een goede reward functie te definiëren
- Exploitatie-Exploratie trade-off
- _Credit assignment_: Uitdagend om wiskundig te bepalen welke acties verantwoordelijk waren voor _rewards_ (of het gebrek daaraan)
- Training stability: RL training kan heel instabiel zijn
- _Sparse rewards_: Bij veel problemen krijg je zelden feedback